In [7]:
'''
This spectrum extraction pipeline is adapted by sas pipeline
https://www.cosmos.esa.int/web/xmm-newton/sas-thread-esasspec
'''
import numpy as np
import os
from glob import glob


# Set SAS environment
define workdir

In [1]:
workdir  = '/stage/headat/yanling/xmm/reduction_230227/spectras'
os.makedirs(workdir, exist_ok=True)
datadir = '/stage/headat/yanling/xmm/reduction_230227/101'
figdir = 
# input_evtfile = f'{instrument}_hiband_loband_clean_{today}_fov.fits'


In [9]:
obs = 101
f = open(f'{workdir}/set_env_{obs}.sh', 'w')
string = f'''
sasversion
export SAS_CCF=/stage/headat/yanling/xmm/0502020{obs}/ccf.cif
export SAS_ODF={glob(f'/stage/headat/yanling/xmm/0502020{obs}/*0502020{obs}_*SUM.SAS')[0]}
sasversion
'''
f.write(string)
f.close()

# generate vignetted corrected, bkg subtracted images

In [ ]:
# get emosaic images (do it locally when extracting sb profile, and upload to sron server)
# also define spectra extraction region locally and upload to server
# regions should be in detector coordinates


# extract spectra

extract separate spectra, rmf and arf
from data events and background events

arfgen cross talk?

combine spectra
https://www.cosmos.esa.int/web/xmm-newton/sas-thread-epic-merging

In [ ]:
 epicspeccombine pha="src_spectrum_01.ds src_spectrum_02.ds src_spectrum_03.ds"\
   bkg="bkg_spectrum_01.ds bkg_spectrum_02.ds bkg_spectrum_03.ds" \
   rmf="response_01.rmf response_02.rmf response_03.rmf" \
   arf="arf_01.arf arf_02.arf arf_03.arf" \
   filepha="src_spectrum_grp.ds" \
   filebkg="bkg_spectrum_grp.ds" \
   filersp="response_grp.rmf" \
   allowHEdiff=yes